In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

df = pd.read_csv('/content/Beauty_Health.csv')
df200 = df

stop_words_arabic = ['في', 'من', 'إلى', 'على', 'أن', 'ما', 'و', 'عن', 'أو', 'هذا', 'كل', 'بين','الله','جدا']

def remove_stop_words(text):
    words = text.split()
    return ' '.join([word for word in words if word not in stop_words_arabic])

df200['comment_cleaned'] = df200['comment'].apply(remove_stop_words)

vectorizer = CountVectorizer(stop_words=stop_words_arabic)
X = vectorizer.fit_transform(df200['comment_cleaned'])

lda = LatentDirichletAllocation(n_components=8, random_state=42)
lda.fit(X)

def get_topic_name(model, n_words=6):
    feature_names = vectorizer.get_feature_names_out()
    topic_names = []
    for topic_idx, topic in enumerate(model.components_):
        top_words_idx = topic.argsort()[:-n_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topic_name = f"{topic_idx+1}_" + "_".join(top_words[:n_words])
        topic_names.append(topic_name)
    return topic_names

def get_all_sentences_for_topic(model, X, texts, ids):
    topic_results = model.transform(X)
    all_sentences = []
    topic_names = get_topic_name(model)

    for idx in range(len(texts)):
        max_score_idx = np.argmax(topic_results[idx])
        max_score = topic_results[idx][max_score_idx]

        all_sentences.append({
            "Topic": f"Topic {max_score_idx + 1}",
            "Count": np.sum(topic_results[:, max_score_idx] > 0.1),
            "Name": f"Topic {max_score_idx + 1}",
            "CustomName": topic_names[max_score_idx],
            "Representation": ', '.join([vectorizer.get_feature_names_out()[i] for i in model.components_[max_score_idx].argsort()[:-10 - 1:-1]]),
            "Representative_Docs": texts.iloc[idx],
            "comment_cleaned": ids.iloc[idx],
            "Score": max_score
        })

    return pd.DataFrame(all_sentences)

df_all_sentences = get_all_sentences_for_topic(lda, X, df200['comment'], df200['review_id'])

df200['topic'] = np.argmax(lda.transform(X), axis=1)

#df200.to_csv('data_with_topics2.csv', index=False)


In [ ]:
df_all_sentences.head(40)

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score
0,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",غلطتي ما جيت شفت تقيمه هنا في معروف تاخير وماط...,360948.0,0.941611
1,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسواء براند واسواء خدمه عملاء محد يرد عليك تاخ...,356339.0,0.937469
2,Topic 4,2064,Topic 4,4_سريع_تعامل_راقي_جوده_رائع_افضل,"سريع, تعامل, راقي, جوده, رائع, افضل, توصيل, وس...",اسوء خدمه عملاء تاخير في توصيل الطلب وعدم الرد...,353856.0,0.576173
3,Topic 5,1690,Topic 5,5_العبايه_العبايات_مره_شي_القماش_تجن,"العبايه, العبايات, مره, شي, القماش, تجن, الي, ...",اسواء متجر لايوجد اي مصداقيه طلبت من عشر ايام ...,345234.0,0.655908
4,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, من...",تاخير في الطلبيه,338152.0,0.708231
5,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, من...",استخدام شركه شحن سيئه,338152.0,0.622277
6,Topic 8,1790,Topic 8,8_اخر_تعامل_مره_مو_اول_شاء,"اخر, تعامل, مره, مو, اول, شاء, ان, اطلب, باذن,...",التاخير في الرد على الوتساب,338152.0,0.780978
7,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, من...",رد غير مهني او احترافي في الوت ساب الطلبيه ال...,338152.0,0.511616
8,Topic 8,1790,Topic 8,8_اخر_تعامل_مره_مو_اول_شاء,"اخر, تعامل, مره, مو, اول, شاء, ان, اطلب, باذن,...",اسوء متجر وصلني العطر مكسور وفيه نص الكيه,332615.0,0.416364
9,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, من...",وخمه عملاء سيئه جدا من شهر وزياده ما ردوا على ...,332615.0,0.902679


In [ ]:
df_all_sentences.head(40)

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score
0,Topic 2,1734,Topic 2,2_متجر_خدمه_افضل_التعامل_شكرا_جميل,"متجر, خدمه, افضل, التعامل, شكرا, جميل, رائع, م...",غلطتي ما جيت شفت تقيمه هنا في معروف تاخير وماط...,360948.0,0.945081
1,Topic 2,1734,Topic 2,2_متجر_خدمه_افضل_التعامل_شكرا_جميل,"متجر, خدمه, افضل, التعامل, شكرا, جميل, رائع, م...",اسواء براند واسواء خدمه عملاء محد يرد عليك تاخ...,356339.0,0.549622
2,Topic 4,4429,Topic 4,4_تعامل_راقي_رائع_سريع_التعامل_وسرعه,"تعامل, راقي, رائع, سريع, التعامل, وسرعه, جميل,...",اسوء خدمه عملاء تاخير في توصيل الطلب وعدم الرد...,353856.0,0.543872
3,Topic 5,2338,Topic 5,5_العبايه_العبايات_تجن_يجن_شي_الي,"العبايه, العبايات, تجن, يجن, شي, الي, القماش, ...",اسواء متجر لايوجد اي مصداقيه طلبت من عشر ايام ...,345234.0,0.504171
4,Topic 3,2417,Topic 3,3_ولا_طلبت_مره_وصلني_لي_نفس,"ولا, طلبت, مره, وصلني, لي, نفس, الطلب, الي, من...",تاخير في الطلبيه,338152.0,0.732519
5,Topic 2,1734,Topic 2,2_متجر_خدمه_افضل_التعامل_شكرا_جميل,"متجر, خدمه, افضل, التعامل, شكرا, جميل, رائع, م...",استخدام شركه شحن سيئه,338152.0,0.836426
6,Topic 4,4429,Topic 4,4_تعامل_راقي_رائع_سريع_التعامل_وسرعه,"تعامل, راقي, رائع, سريع, التعامل, وسرعه, جميل,...",التاخير في الرد على الوتساب,338152.0,0.793058
7,Topic 3,2417,Topic 3,3_ولا_طلبت_مره_وصلني_لي_نفس,"ولا, طلبت, مره, وصلني, لي, نفس, الطلب, الي, من...",رد غير مهني او احترافي في الوت ساب الطلبيه ال...,338152.0,0.602361
8,Topic 3,2417,Topic 3,3_ولا_طلبت_مره_وصلني_لي_نفس,"ولا, طلبت, مره, وصلني, لي, نفس, الطلب, الي, من...",اسوء متجر وصلني العطر مكسور وفيه نص الكيه,332615.0,0.626063
9,Topic 3,2417,Topic 3,3_ولا_طلبت_مره_وصلني_لي_نفس,"ولا, طلبت, مره, وصلني, لي, نفس, الطلب, الي, من...",وخمه عملاء سيئه جدا من شهر وزياده ما ردوا على ...,332615.0,0.909796


In [ ]:
df_all_sentences['Topic'].value_counts()

,count
Topic,
Topic 6,1483
Topic 4,1370
Topic 3,1061
Topic 8,979
Topic 5,970
Topic 2,799
Topic 7,784
Topic 1,530


In [ ]:
df_all_sentences[df_all_sentences['Topic'] == 'Topic 7']

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score
12,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",لو فيه تقيم اقل من نجمه حطيت طلبت قبل رمضان وك...,331023.0,0.529196
25,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",لا يتم الرد على الرسائل والاستفسارات,326447.0,0.450770
49,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",اسوء متجر مكن تشوفونه كذب وريحه خايسه جدا واي ...,352596.0,0.536133
64,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",انتبهو ياناس منهم تراهم اداره فاشله استرجعت عط...,330347.0,0.575570
121,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",العبايه كويسه ما شاء الله واسعارها جدا جميله ...,161343.0,0.699445
...,...,...,...,...,...,...,...,...
7893,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",توفقت في الفساتين لي ولامي ومره اعجبتنا والله ...,221083.0,0.804691
7924,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",رعه وصلني الطلب بنفس الوقت المحد الفستان جدا ج...,182727.0,0.368429
7952,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",موقع جدا رائع واسعارهم حلوه ومنتجاتهم جميله وج...,193640.0,0.440538
7958,Topic 7,1324,Topic 7,7_جميله_حلوه_العافيه_العبايه_تجربه_متازه,"جميله, حلوه, العافيه, العبايه, تجربه, متازه, م...",ماشاله من اصدق المتاجر يلي اتعامل معها و من ار...,193603.0,0.699768


In [ ]:
Topic_2 = df_all_sentences[df_all_sentences['Topic'] == 'Topic 2']
Topic_2

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score
0,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",غلطتي ما جيت شفت تقيمه هنا في معروف تاخير وماط...,360948.0,0.941611
1,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسواء براند واسواء خدمه عملاء محد يرد عليك تاخ...,356339.0,0.937469
13,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسواء متجر عطور طلبت عطر وما عجبني وابغا ارجعه...,329987.0,0.969795
14,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسوء متجر تشترون منه عطور,329400.0,0.854135
24,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",خدمه عملاء سيئه,326447.0,0.781089
...,...,...,...,...,...,...,...,...
7872,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",خدمه سريعه و اسعار منافسه و الحساب يقدم معلوما...,196771.0,0.540171
7873,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",خدمات جميله وتعامل رائع يقدم كل الخدمات الي تح...,171616.0,0.961908
7894,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",لامام عزيزتي,221083.0,0.708129
7936,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",كاتجربه متجر رائع جدا والمنتجات لا يعلا عليها ...,194117.0,0.677090


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
Topic_2['Topic_value'] = 'التوصيل'
Topic_2

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score,Topic_value
0,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",غلطتي ما جيت شفت تقيمه هنا في معروف تاخير وماط...,360948.0,0.941611,التوصيل
1,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسواء براند واسواء خدمه عملاء محد يرد عليك تاخ...,356339.0,0.937469,التوصيل
13,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسواء متجر عطور طلبت عطر وما عجبني وابغا ارجعه...,329987.0,0.969795,التوصيل
14,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",اسوء متجر تشترون منه عطور,329400.0,0.854135,التوصيل
24,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",خدمه عملاء سيئه,326447.0,0.781089,التوصيل
...,...,...,...,...,...,...,...,...,...
7872,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",خدمه سريعه و اسعار منافسه و الحساب يقدم معلوما...,196771.0,0.540171,التوصيل
7873,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",خدمات جميله وتعامل رائع يقدم كل الخدمات الي تح...,171616.0,0.961908,التوصيل
7894,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",لامام عزيزتي,221083.0,0.708129,التوصيل
7936,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اك...",كاتجربه متجر رائع جدا والمنتجات لا يعلا عليها ...,194117.0,0.677090,التوصيل


In [ ]:
Topic_2.sample(20)

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score,Topic_value
5140,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",صراحه جربت عطور من عندهم الاسعار رخيصه والراحه فريده من نوعها صادقين الله يوفقهم فيه عطرين ما ناسبوني وكلمت الدعم وردولي المبلغ والعطر بقى عندي وتوصيلهم سريع ب ايام,318173.0,0.673385,التوصيل
6327,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",متجر متنوع وجدا جميل واسعاره معقوله وسرعه في التوصيل ودقه في التغليف وجوده عاليه الله يبارك لي صاحبه,335004.0,0.330842,التوصيل
5727,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",متجر رائع و ممتاز,346158.0,0.781076,التوصيل
1619,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",تكنيك جديد و رائع,240118.0,0.449953,التوصيل
7862,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",متجر جميل ومتكامل,243783.0,0.781143,التوصيل
4891,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",مشاله متجر عبايات مرتبه وانيقه وسعرها جدا مناسب طلبت منهم مرتين ولاغلطه تبارك الله,321866.0,0.472697,التوصيل
3962,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",هذا المتجر ممتازه جودتهم انصح فيه,359076.0,0.853994,التوصيل
1553,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",افضل رموش بالشرقيه,260460.0,0.781162,التوصيل
2988,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",شكرا لكم خامه الفستان جدا رائعه وسرعه التوصيل وسهوله التعامل جدا رائعه افضل متجر تعملت معه,291731.0,0.546735,التوصيل
615,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",لا زلت استخدم التراد الخفيف لتعطير المنزلي اليومي,170573.0,0.902752,التوصيل


In [ ]:
indices_to_select = [0, 1 ,13 , 59 , 82 ,162 , 164, 262 , 286 ,293	, 275 , 269 , 228 ,302 , 2212 , 2988, 5140 , 6327,7872 , 7936 ]
new_Topic_2 = Topic_2.loc[indices_to_select]
new_Topic_2

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score,Topic_value
0,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",غلطتي ما جيت شفت تقيمه هنا في معروف تاخير وماطله وتعامل سيء لابعد حدود الله لا يسامحهم,360948.0,0.941611,التوصيل
1,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",اسواء براند واسواء خدمه عملاء محد يرد عليك تاخير ف الطلب ومحد يرد عليك,356339.0,0.937469,التوصيل
13,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",اسواء متجر عطور طلبت عطر وما عجبني وابغا ارجعه اكلم الدعم الفني ما يردون اسبوعين والى الان ما ردو والله انصح الي ناوي يشتري يهون ويشتري من متجر ثاني المتجر جدا جدا سيء,329987.0,0.969795,التوصيل
59,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",انصح بعدم التعامل معهم لايوجد مصداقيه وتاخير في الارسال بالاسابيع والدعم مايرد وحسبنا الله ونعم الوكيل,333787.0,0.941611,التوصيل
82,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",سوء في خدمه العملاء وعدم مصداقيه في التوصيل,328984.0,0.604922,التوصيل
162,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",جوده الاقمشه رائعه والخياطه مره مرتبه والسعر جدا جميل وفي متناول الجميع خدمه الشحن سريعه والتعامل اكثر من رائع انصح الكل بتعامل معها,92653.0,0.389768,التوصيل
164,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",خدمه سريعه ومنتجات رائعه,92448.0,0.824924,التوصيل
262,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",الحساب جميل ومتنوع والعدسات رائعه وجميعها متوفره وسرعه بالتوصيل وروعه التغليف,150430.0,0.507715,التوصيل
286,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",اشتريت منهم عطر وكان رائع واسعارهم منافسه واهم شي التوصيل كان سريع,367599.0,0.704577,التوصيل
293,Topic 2,1378,Topic 2,2_متجر_خدمه_رائع_افضل_سريعه_جميل,"متجر, خدمه, رائع, افضل, سريعه, جميل, رائعه, اكثر, متازه, مع",متجر موثوق وعطورهم اصليه وتعاملهم مره ممتاز وسرعه التوصيل رهيبه شكرا لكم,366979.0,0.572721,التوصيل


In [ ]:
new_Topic_2.shape

(20, 9)

In [ ]:
new_Topic_2.to_csv("Topic_2.csv", index=False)

In [ ]:
Topic_3 = df_all_sentences[df_all_sentences['Topic'] == 'Topic 3']
Topic_3

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score
4,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",تاخير في الطلبيه,338152.0,0.708231
5,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",استخدام شركه شحن سيئه,338152.0,0.622277
7,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",رد غير مهني او احترافي في الوت ساب الطلبيه الى الان لها اكثر من يوم لم تصل,338152.0,0.511616
9,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",وخمه عملاء سيئه جدا من شهر وزياده ما ردوا على تواصلي معهم,332615.0,0.902679
11,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",خدمه العملاء سيئه مره لي شهر اتواصل معهم ولا احد رد علي,331689.0,0.816534
...,...,...,...,...,...,...,...,...
7949,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",سرعه التوصيل والشحن,193672.0,0.780966
7950,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",وصول الطلب نفس طلبنا وزياده تغليف الطلبيه تغليف ميز مجانا,193672.0,0.920373
7951,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",افضل موسه اشتغله معها وصرت واثق بالمنتجات كثير وكل العملاء الي تعامله معهم مستحيل ما يرجعون مره ثاني لان جوده المنتج جدا جميله واصليه واسعار مناسبه,193665.0,0.534142
7959,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",من افضل موسات التجميل الي تعاملت معهم منظمين بكل شي منتجاتهم موثوقه مهتمين بتغليف وترتيب المنتجات وقت شحن الطلب ذوق في التعامل مع المسوقات و الزبائن بالمختصر موسه مثاليه ماعليها ولا غلطه حقيقي التعليق دا وصف الموقع وانصفه بالتمام كاملين من جميع الجهات ربي يزيدهم ويبارك لهم,193584.0,0.658138


In [ ]:
Topic_3['Topic_value'] = 'المصداقيه'
Topic_3

,Topic,Count,Name,CustomName,Representation,Representative_Docs,ID,Score,Topic_value
4,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",تاخير في الطلبيه,338152.0,0.708231,المصداقيه
5,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",استخدام شركه شحن سيئه,338152.0,0.622277,المصداقيه
7,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",رد غير مهني او احترافي في الوت ساب الطلبيه الى الان لها اكثر من يوم لم تصل,338152.0,0.511616,المصداقيه
9,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",وخمه عملاء سيئه جدا من شهر وزياده ما ردوا على تواصلي معهم,332615.0,0.902679,المصداقيه
11,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",خدمه العملاء سيئه مره لي شهر اتواصل معهم ولا احد رد علي,331689.0,0.816534,المصداقيه
...,...,...,...,...,...,...,...,...,...
7949,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",سرعه التوصيل والشحن,193672.0,0.780966,المصداقيه
7950,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",وصول الطلب نفس طلبنا وزياده تغليف الطلبيه تغليف ميز مجانا,193672.0,0.920373,المصداقيه
7951,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",افضل موسه اشتغله معها وصرت واثق بالمنتجات كثير وكل العملاء الي تعامله معهم مستحيل ما يرجعون مره ثاني لان جوده المنتج جدا جميله واصليه واسعار مناسبه,193665.0,0.534142,المصداقيه
7959,Topic 3,1867,Topic 3,3_ولا_طلبت_مره_وصلني_الفستان_نفس,"ولا, طلبت, مره, وصلني, الفستان, نفس, الطلب, منهم, التوصيل, الي",من افضل موسات التجميل الي تعاملت معهم منظمين بكل شي منتجاتهم موثوقه مهتمين بتغليف وترتيب المنتجات وقت شحن الطلب ذوق في التعامل مع المسوقات و الزبائن بالمختصر موسه مثاليه ماعليها ولا غلطه حقيقي التعليق دا وصف الموقع وانصفه بالتمام كاملين من جميع الجهات ربي يزيدهم ويبارك لهم,193584.0,0.658138,المصداقيه
